In [1]:
import os
import glob
import pandas as pd
import shutil
from PIL import Image

In [2]:
input_dir = "./image"
output_train_dir = "./train"
output_test_dir = "./test"
extension = "png"  # jpg

In [3]:
# dataframes
tran = pd.read_csv("label_tran.csv")
redundant = pd.read_csv("redundant.csv")
exclude = pd.read_csv("exclude.csv")

actl = tran.copy().sort_values(
    "update", ascending=False
).drop_duplicates(
    "id", keep="first"
).drop(
    "update", axis=1
).reset_index(drop=True)

actl = actl.merge(redundant[["id", "master_id"]].drop_duplicates("id"), how="left")
actl = actl[actl["master_id"].isnull()].drop("master_id", axis=1)

exclude["master_id"] = 1
actl = actl.merge(exclude[["id", "master_id"]].drop_duplicates("id"), how="left")
actl = actl[actl["master_id"].isnull()].drop("master_id", axis=1)

labels = actl.reset_index(drop=True)

In [4]:
# augumentation setting
alpha = labels["like"].mean()
times = min(int((1-alpha)/alpha), 10)  # 正例が少ないものとして比率を算出
n = 800  # 水増し画像は(n,n)で加工したのち(224,224)として出力される
dig = 15  # 水増し画像は(-dig,dig)の範囲で回転加工される
idigre = dig*2/(times-1)  # 水増し画像の回転角度

In [5]:
times

10

In [6]:
# outputディレクトリの指定
path_list = [f"{output_train_dir}/00_notlike/",
             f"{output_train_dir}/01_like/",
             f"{output_test_dir}/"]

# フォルダを作成
for path in path_list:
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        shutil.rmtree(path)
        os.makedirs(path)

In [ ]:
%%time
# 加工済ファイル出力
# train
for i, l in zip(labels["id"], labels["like"]):
    img = Image.open(f'{input_dir}/{i}.{extension}')
    img = img.convert("RGB")
    w, h = img.size
    if w/h > 1:
        img = img.crop((w/6, 0, w/2, h))
        # img = img.crop((w/2, 0, w, h))
        w = w/2
    if (times > 1) & (l == 1):
        for j in range(times):
            digree = idigre*j - dig
            img.rotate(digree).resize((n, n)).crop(
                (int(0.1*n), int(0.1*n), int(0.9*n), int(0.9*n))
            ).resize((224, 224)).save(path_list[int(l)] + f"{i}.{j}.png")
    img = img.resize((224, 224))
    img.save(path_list[int(l)] + f"{i}.png")

In [ ]:
# test id 取得
exist = pd.DataFrame({
    "png": labels["id"].apply(lambda x: f"{input_dir}/{x}.{extension}"),
    "exist": "1"
}).drop_duplicates()
allpng = pd.DataFrame({
    "png": glob.glob(f"{input_dir}/*")
}).drop_duplicates().merge(exist, on="png", how="left")
allpng["id"] = allpng["png"].apply(lambda x : int(x.split("/")[-1].split(".")[0]))
allpng = allpng[
    allpng.merge(
        exclude[["id", "master_id"]].drop_duplicates("id"), on="id", how="left"
    )["master_id"].isnull()
].drop("id", axis=1).reset_index(drop=True)
test_id = list(allpng[allpng["exist"].isnull()]["png"].apply(
    lambda x: int(x.split(f".{extension}")[0].split("/")[-1])))

In [ ]:
%%time
# 加工済ファイル出力
# test
for i in test_id:
    img = Image.open(f'{input_dir}/{i}.{extension}')
    img = img.convert("RGB")
    w, h = img.size
    if w/h > 1:
        img = img.crop((w/6, 0, w/2, h))
        # img = img.crop((w/2, 0, w, h))
        w = w/2
    img = img.resize((224, 224))
    img.save(path_list[2] + f"{i}.png")